In [1]:
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
directors = []
mpaaRatings = []
X = [] # This holds the movie data that the model will use to train.
y = [] # This holds whether the movies won an oscar or not that the model will use to train.

dummyData = pd.read_csv("oscarsDummyData.csv", encoding = "ISO-8859-1")
n = len(dummyData.index)

for i in range(0, n):
    movie = list(dummyData.iloc[i])
    director = movie[7]
    mpaaRating = movie[10]
    
    if not(director in directors):
        directors.append(director)
    movie[7] = directors.index(director) # replace director to its unique numerical value
    
    if not(mpaaRating in mpaaRatings):
        mpaaRatings.append(mpaaRating)
    movie[10] = mpaaRatings.index(mpaaRating) # replace mpaa rating to its unique numerical value
    
    X.append(movie[3:-1])
    
    if movie[1]:
        y.append(1)
    else:
        y.append(0)
        
# train_test_split takes the datasets, shuffles the datasets, and splits it into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, stratify = y)

In [9]:
svc = SVC(probability=True)
svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
# This predict function produces yes or no Oscar predictions.
predictions = svc.predict(X_test)
pretty_list = "\n".join("{}\t\t{}".format(x, y) for x, y in zip(predictions, y_test))
print("Predictions\tActual")
print(pretty_list)

Predictions	Actual
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		1
0		0
0		0
0		1
0		1
0		0
0		0
0		1


In [11]:
# This predict_proba function produces the probability of the movies winning an oscar.
predictions = svc.predict_proba(X_test)[:,1]
pretty_list = "\n".join("{:.2f}\t{}".format(x, y) for x, y in zip(predictions, y_test))
print("% of 1 \tActual")
print(pretty_list)

% of 1 	Actual
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	0
0.14	1
0.14	0
0.14	0
0.14	1
0.14	1
0.14	0
0.14	0
0.14	1
